<a href="https://colab.research.google.com/github/straxFromIbr/UnetModelScript/blob/main/home/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
% cd /content
from google.colab import drive
drive.mount('/content/drive')
! unzip /content/drive/MyDrive/卒研/mass_roads256.zip &>/dev/null
! git clone --depth 1 https://github.com/straxFromIbr/UnetModelScript/
% cd /content/UnetModelScript/home
! pip install tensorflow-addons scikit-image &>/dev/null  

/content
Mounted at /content/drive
Cloning into 'UnetModelScript'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 28 (delta 0), reused 11 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), done.
/content/UnetModelScript/home


In [3]:
import random
import math
import pathlib

import train
from model import losses

BATCH_SIZE = 64

In [4]:
random.seed(1)
datadir = '/content/mass_roads256/train'
suffix = 'png'
pathlist = (pathlib.Path(datadir) / "map").glob(f"*.{suffix}")
pathlist = list(pathlist)
random.shuffle(pathlist)

In [5]:
dsrate = 1.0
nb_tr = int(len(pathlist) * 0.8 * dsrate)
nb_va = int(len(pathlist) * 0.2)
tr_pathlist = pathlist[:nb_tr]
va_pathlist = pathlist[nb_tr : nb_tr + nb_va]

steps_per_epoch = math.ceil(nb_tr / BATCH_SIZE)
va_steps = math.ceil(nb_va / BATCH_SIZE)

ds_root = pathlib.Path(datadir)
tr_sat_pathlist = sorted([str(ds_root / "sat" / path) for path in tr_pathlist])
tr_map_pathlist = sorted([str(ds_root / "map" / path) for path in tr_pathlist])

In [6]:
train_ds = train.mkds(
    tr_sat_pathlist, 
    tr_map_pathlist, 
    batch_size=BATCH_SIZE, 
    aug=True,
    zoom=True,
    flip=True,
    rotate=True,
    cutmix=True,
    nbmix=4,
)

In [7]:
va_sat_pathlist = sorted([str(ds_root / "sat" / path) for path in va_pathlist])
va_map_pathlist = sorted([str(ds_root / "map" / path) for path in va_pathlist])
valid_ds = train.mkds(va_sat_pathlist, va_map_pathlist, batch_size=BATCH_SIZE, test=True)

In [8]:
loss = losses.DICELoss("DICE")
model = train.compile_model(loss=loss)

In [ ]:
model_history = model.fit(
     train_ds,
     epochs=20,
     validation_data=valid_ds,
     steps_per_epoch=steps_per_epoch,
     validation_steps=va_steps,
)

Epoch 1/20
  1/108 [..............................] - ETA: 29:04 - loss: 0.9002 - mean_io_u: 0.4995